In [1]:
import datastore

import pandas as pd
import matplotlib.pyplot as plt

spot = datastore.getSpotPrice()

total_consumption_production = datastore.getTotalConsumptionProduction()

production_se1_mwh = datastore.getAllSE1ProductionMWh() 
production_se2_mwh = datastore.getAllSE2ProductionMWh()
production_se3_mwh = datastore.getAllSE3ProductionMWh()
production_se4_mwh = datastore.getAllSE4ProductionMWh()

transTo = datastore.getTransmissionCapTo()
transFrom = datastore.getTransmissionCapFrom()

# netFlow = datastore.getNetFlow() # Too many missing values
flow = datastore.getFlow()

capTo = datastore.getFlowCapacityTo()
capFrom = datastore.getFlowCapacityFrom()

exchangeFrom = datastore.getExchangeFrom()
exchangeTo = datastore.getExchangeTo() 

turnOver = datastore.getTurnover()

temp_pen = datastore.getDailyWeather() # DAILY celsius
wind_velocities = datastore.getHourlyWindVelocity() # m/s
hydroReservoirs = datastore.getWeeklyHydroReservs() # GWh


Missing values in Index(['spotPrice'], dtype='object') : 6
Missing values in Index(['ConsumptionGWh', 'ProductionGWh'], dtype='object') : 14
Missing values in Index(['waterSE1(MWh)', 'windSE1(MWh)', 'trmSE1MWh'], dtype='object') : 46
Missing values in Index(['waterSE2(MWh)', 'windSE2(MWh)', 'solSE2MWh', 'trmSE2MWh', 'OthSE2MWh'], dtype='object') : 144
Missing values in Index(['waterSE3(MWh)', 'windSE3(MWh)', 'nucSE3(MWh)', 'solSE3MWh',
       'trmSE3MWh', 'OthSE3MWh'],
      dtype='object') : 139
Missing values in Index(['waterSE4(MWh)', 'windSE4(MWh)', 'solSE4MWh', 'trmSE4MWh', 'OthSE4MWh'], dtype='object') : 155
Missing values in Index(['TDK1SE3', 'TNO1SE3', 'TSE2SE3', 'TF1SE3', 'TSE4SE3'], dtype='object') : 0
Missing values in Index(['TSE3DK1', 'TSE3NO1', 'TSE3SE2', 'TSE3F1', 'TSE3SE4'], dtype='object') : 0
Missing values in Index(['F_LOWSE2SE3', 'F_LOWSE3FI', 'F_LOWSE3SE4'], dtype='object') : 6289
Missing values in Index(['C_SE4SE3', 'C_DK1SE3', 'C_FISE3', 'C_NO1SE3', 'C_SE2SE3'], 

In [67]:
import calendar_features as cf

dataset = pd.concat([spot, total_consumption_production,
                     production_se1_mwh, production_se2_mwh, production_se3_mwh, production_se4_mwh, 
                     transTo, transFrom, 
                     flow, capTo, capFrom, exchangeFrom, exchangeTo,
                     turnOver, wind_velocities, hydroReservoirs, temp_pen], axis=1)

dataset.interpolate(method = 'linear', limit_direction = 'forward', inplace=True, axis=0)

# daylight_features = cf.daylight_extractor(dataset)
holiday_features = cf.get_holidays(dataset)
dataset = cf.calendar_transformer(dataset)

dataset = pd.concat([dataset, holiday_features], axis=1)
dataset = pd.get_dummies(dataset, columns=["year", "month", "day_of_week", "hour"])

# add index 
dataset["time_idx"] = range(len(dataset))
dataset["time_idx"] = dataset["time_idx"].astype(int)
dataset["group_ids"] = "SE3" # add dummy variable for group_ids, only one called "SE3"



dataset.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 54024 entries, 2018-01-01 00:00:00 to 2024-02-29 23:00:00
Columns: 115 entries, spotPrice to group_ids
dtypes: float64(61), int32(2), int64(1), object(1), uint8(50)
memory usage: 31.4+ MB


In [37]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

def create_sliding_window(data, sequence_length, stride=1):
    X_list, y_list = [], []
    for i in range(len(data)):
      if (i + sequence_length) < len(data):
        X_list.append(data.iloc[i:i+sequence_length:stride, :].values)
        y_list.append(data.iloc[i+sequence_length, 0])
    return np.array(X_list), np.array(y_list)
  

x = dataset.drop('spotPrice', axis=1)
y = dataset['spotPrice']

x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

train_split = '2021-12-31 23:00:00'
val_split = '2022-12-31 23:00:00'
test_split = '2024-02-29 23:00:00'

train_split_index = dataset.index.get_loc(train_split)
val_split_index = dataset.index.get_loc(val_split)
test_split_index = dataset.index.get_loc(test_split)

x_train = x[:train_split_index]
y_train = y[:train_split_index]

x_val = x[train_split_index:val_split_index]
y_val = y[train_split_index:val_split_index]

x_test = x[val_split_index:test_split_index]
y_test = y[val_split_index:test_split_index]

x_scaler.fit(x_train)
y_scaler.fit(y_train.values.reshape(-1, 1))

x_train = x_scaler.transform(x_train)
y_train = y_scaler.transform(y_train.values.reshape(-1, 1))

x_val = x_scaler.transform(x_val)
y_val = y_scaler.transform(y_val.values.reshape(-1, 1))

x_test = x_scaler.transform(x_test)
y_test = y_scaler.transform(y_test.values.reshape(-1, 1))



# Combine x_train and y_train into a single pandas dataframe
train_df = pd.concat([pd.DataFrame(x_train), pd.DataFrame(y_train, columns=["spotPrice"])], axis=1)
val_df = pd.concat([pd.DataFrame(x_val), pd.DataFrame(y_val, columns=["spotPrice"])], axis=1)
test_df = pd.concat([pd.DataFrame(x_test), pd.DataFrame(y_test, columns=["spotPrice"])], axis=1)

x_train, y_train = create_sliding_window(train_df, 240)
x_val, y_val = create_sliding_window(val_df, 240)
x_test, y_test = create_sliding_window(test_df, 240)


In [68]:
from pytorch_forecasting import TimeSeriesDataSet
 
max_prediction_length = 24 * 5
max_encoder_length = 24 * 5 * 2

training_cutoff = dataset["time_idx"].max() - max_prediction_length

target = "spotPrice"

static_categoricals = ["group_ids"]

time_varying_known_categoricals = dataset.iloc[:, 61:113].columns.to_list()
dataset[time_varying_known_categoricals] = dataset[time_varying_known_categoricals].astype(str).astype("category")



time_varying_known_reals = ["time_idx", "PrecipitationEnergySE(day)", "Temperature(day)", "HydroRes(GWh_week)", "Wind(Pite)"]
time_varying_unknown_reals = dataset.iloc[:, 0:57].columns.to_list()

features = dataset.drop(columns=[target, "time_idx"]).columns.to_list()

training = TimeSeriesDataSet(
    dataset[lambda x: x.time_idx <= training_cutoff],
    
    time_idx="time_idx",
    
    target=target,
    
    group_ids= static_categoricals,
    
    min_encoder_length=max_encoder_length,  
    max_encoder_length=max_encoder_length,
    
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    
    static_categoricals=static_categoricals,
    static_reals=[],
    
    time_varying_known_categoricals= [],
    time_varying_known_reals= [],
    
    time_varying_unknown_categoricals = [],
    time_varying_unknown_reals= time_varying_unknown_reals,
    
    add_relative_time_idx=False,
    add_target_scales=False,
    add_encoder_length=False
)


In [69]:
train = training.to_dataloader(train=True, batch_size=128, shuffle=False)

In [70]:
from LSTM import BayesianLSTM
from torch import nn
import torch

n_features = x_train.shape[1]

batch_size = 128
n_epochs = 150
learning_rate = 0.01

model = BayesianLSTM(n_features=n_features, output_length= 120, batch_size=batch_size)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
model.train()

for e in range(1, n_epochs+1):
    for X_batch, y_batch in train:

        output = model(X_batch)
        loss = criterion(output, y_batch)  
        optimizer.zero_grad() 
        loss.backward()
        optimizer.step()        

    if e % 10 == 0:
      print('epoch', e, 'loss: ', loss.item())